In [2]:
import json
import backoff
import inspect
import asyncio

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import boto3
import os

region = os.environ.get("AWS_REGION")
bedrock_service = boto3.client(
    service_name='bedrock',
    region_name=region,
)

boto3_bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name=region,
)

In [16]:
target_model_name = "anthropic.claude-3-haiku-20240307-v1:0"
target_model_native_version = "bedrock-2023-05-31"
target_model_config = {
    "temperature": 0, "max_output_tokens": 1000, "top_p":0
    }
review_model_name = "anthropic.claude-3-haiku-20240307-v1:0"
review_model_config = {
    "temperature": 0, "max_output_tokens": 10, "top_p":0
    }
review_prompt_template_path = 'review_prompt_template.txt'

In [6]:
# Load the review prompt from the text file
with open(review_prompt_template_path, 'r') as f:
    review_prompt_template = f.read().strip()
            
print(review_prompt_template)

You are a review model tasked with evaluating the correctness of a response to a navigation problem. 
The response may contain detailed steps and explanations, but the final answer is the key point. 
Please determine if the final answer provided in the response is correct based on the ground truth number. 
Respond with 'True' if the final answer is correct and 'False' if it is not. 
Only respond with 'True' or 'False', nothing else.

Model Response:
{model_response}

Ground Truth:
{ground_truth}


In [7]:
prompt="Solve the given problem about geometric shapes"
question='This SVG path element <path d="M 66.84,13.67 L 81.50,22.51 M 81.50,22.51 L 75.76,32.04 L 61.09,23.20 L 55.35,32.73 L 50.26,9.67 L 72.58,4.14 L 66.84,13.67"/> draws a Options: (A) circle (B) heptagon (C) hexagon (D) kite (E) line (F) octagon (G) pentagon (H) rectangle (I) sector (J) triangle'

In [8]:
body = json.dumps({
            "anthropic_version": target_model_native_version,
            "max_tokens": target_model_config['max_output_tokens'],
            "system": prompt,
            "temperature": target_model_config['temperature'],
            "top_p": target_model_config['top_p'],
            "messages": [
                {
                "role": "user",
                "content": [{"type": "text", "text": question}]
                }
            ],
        })


In [9]:
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=target_model_name, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

In [10]:
from IPython.display import clear_output, display, display_markdown, Markdown
display_markdown(Markdown(print(response_body["content"][0]["text"], end='')))

To determine the geometric shape drawn by the given SVG path element, we need to analyze the coordinates and the sequence of the path commands.

The path element starts with the command "M 66.84,13.67", which is a move-to command that sets the starting point of the path. Then, the path continues with the following commands:

1. "L 81.50,22.51": This is a line-to command that draws a line from the previous point to the new point (81.50, 22.51).
2. "L 75.76,32.04": This is another line-to command that draws a line from the previous point to the new point (75.76, 32.04).
3. "L 61.09,23.20": This is another line-to command that draws a line from the previous point to the new point (61.09, 23.20).
4. "L 55.35,32.73": This is another line-to command that draws a line from the previous point to the new point (55.35, 32.73).
5. "L 50.26,9.67": This is another line-to command that draws a line from the previous point to the new point (50.26, 9.67).
6. "L 72.58,4.14": This is another line-to com

In [11]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("lukaemon/bbh", "geometric_shapes", cache_dir="./bbh_nshapes_cache")
data = dataset["test"]
data = data.shuffle(seed=1234)

training = data.select(range(100))
df_train = pd.DataFrame({"question": training["input"], "answer": training["target"]})

test = data.select(range(100, 200))
df_test = pd.DataFrame({"question": test["input"], "answer": test["target"]})

df_train.to_csv("train.csv", index=False)
df_test.to_csv("test.csv", index=False)

In [12]:
print(review_prompt_template_path)

review_prompt_template.txt


In [17]:
df_train=df_test
from prompt_evaluator import PromptEvaluator
try:
    #evaluator = PromptEvaluator(df_train, target_model_name, target_model_config, review_model_name, target_model_native_version, review_model_config, review_prompt_template_path)
    evaluator = PromptEvaluator(df_train, target_model_name, target_model_native_version, target_model_config, review_model_name, review_model_config, review_prompt_template_path)

    print(f"Function signature: {PromptEvaluator.__name__}{inspect.signature(PromptEvaluator)}")
except TypeError as e:
    print(f"Error: {e}")
    print(f"Function signature: {PromptEvaluator.__name__}{inspect.signature(PromptEvaluator)}")
    print(PromptEvaluator.__init__.__code__.co_varnames)

Function parameters:
self = <prompt_evaluator.PromptEvaluator object at 0x7f7a77757b80>
df_train =                                              question answer
0   This SVG path element <path d="M 78.01,21.13 L...    (K)
1   This SVG path element <path d="M 41.70,41.16 A...    (K)
2   This SVG path element <path d="M 73.73,87.22 L...    (J)
3   This SVG path element <path d="M 64.00,63.00 L...    (B)
4   This SVG path element <path d="M 24.45,23.22 L...    (F)
..                                                ...    ...
95  This SVG path element <path d="M 50.91,18.41 L...    (C)
96  This SVG path element <path d="M 26.95,11.95 A...    (K)
97  This SVG path element <path d="M 45.10,9.67 L ...    (J)
98  This SVG path element <path d="M 34.08,19.65 L...    (B)
99  This SVG path element <path d="M 43.79,3.46 L ...    (J)

[100 rows x 2 columns]
target_model_name = anthropic.claude-3-haiku-20240307-v1:0
target_model_native_version = bedrock-2023-05-31
target_model_config = {'temperature':

In [14]:
frame = inspect.currentframe()
args, _, _, values = inspect.getargvalues(frame)
print("Function parameters:")
for arg in args:
    print(f"{arg} = {values[arg]}")

Function parameters:


In [19]:
    prompt = "Solve the given problem about geometric shapes."
    question=df_train.loc[0, 'question']
    print(question)
    await evaluator.main(prompt)
    #await evaluator.generate_target_model_response(question, prompt)

This SVG path element <path d="M 78.01,21.13 L 69.60,22.72 L 77.07,62.39 L 85.49,60.81 L 78.01,21.13"/> draws a
Options:
(A) circle
(B) heptagon
(C) hexagon
(D) kite
(E) line
(F) octagon
(G) pentagon
(H) rectangle
(I) sector
(J) triangle
(K) trapezoid


Evaluating Prompt:   0%|          | 0/100 [00:00<?, ?it/s]

in generate_target_model_response
bedrock-2023-05-31
Target: invoking bedrock body
in generate_review_model_response
review: invoking bedrock body


Evaluating Prompt:   1%|          | 1/100 [00:01<03:12,  1.94s/it]

{'id': 'msg_bdrk_014KTU1vyrS7m667ns6gCYCE', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-haiku-20240307', 'content': [{'type': 'text', 'text': 'True'}], 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 266, 'output_tokens': 4}}
in generate_target_model_response
bedrock-2023-05-31
Target: invoking bedrock body


...

in generate_review_model_response
review: invoking bedrock body
{'id': 'msg_bdrk_01C9gq6D3YZEuRV4RsqNqCxa', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-haiku-20240307', 'content': [{'type': 'text', 'text': 'False'}], 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 286, 'output_tokens': 4}}
in generate_target_model_response
bedrock-2023-05-31
Target: invoking bedrock body


Evaluating Prompt:  99%|█████████▉| 99/100 [04:18<00:02,  2.26s/it]

in generate_review_model_response
review: invoking bedrock body
{'id': 'msg_bdrk_012xTDKsumUTNenARS6HRW7k', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-haiku-20240307', 'content': [{'type': 'text', 'text': 'False'}], 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 262, 'output_tokens': 4}}
in generate_target_model_response
bedrock-2023-05-31
Target: invoking bedrock body
in generate_review_model_response
review: invoking bedrock body


Evaluating Prompt: 100%|██████████| 100/100 [04:20<00:00,  2.61s/it]

{'id': 'msg_bdrk_012DUk7RvqPXqbmEYo9cXjCU', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-haiku-20240307', 'content': [{'type': 'text', 'text': 'False'}], 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 310, 'output_tokens': 4}}
Overall accuracy for the prompt: 0.36
